In [100]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [101]:
DATA_DIR="C:\SUFIYAN\STUDY MATERIALS\FOR CLASS\sagemaker-flights-price-prediction\data"

Function for retrieving Data

In [102]:
def get_data(name):
    file_name=f'{name}.csv'
    path=os.path.join(DATA_DIR,file_name)
    return pd.read_csv(path)

df=get_data('flights')

In [103]:
df

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The dataset contains 10,683 rows and 11 features
- Columns Route and Total_Stops have missing value each
- The data types of some features isn't appropriate

In [105]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


- 1. PREMILINARY ANALYSIS

1.1 Observations

- The Data Type of `Date_of_Journey`, `Arrival_Time` and `Dep_Time` should be datetime
- `Duration` & `Total_Stops` column is mixed, it should be numeric
- There are total 220 duplicated entries, these should be removed

- 1.2 Change Data Types

In [106]:
(
    df
    .loc[df.duplicated(keep=False)]
    .sort_values(['Airline','Date_of_Journey' ,'Source','Destination'])

)
df.loc[np.where(df['Arrival_Time']=='23:55 02 Mar')]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135


- 1.3 Remove Duplicated Entries

In [107]:
df.duplicated().sum()

220

In [108]:
df.duplicated(keep=False)

0        False
1        False
2        False
3        False
4        False
         ...  
10678    False
10679    False
10680    False
10681    False
10682    False
Length: 10683, dtype: bool

- 2 Detailed Analysis

- 2.1 Airline

In [109]:
df.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [110]:
(
    df
    .Airline
    .str.replace("Premium economy","")
    .str.replace("Business","")
    .str.title()
    .unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Vistara ',
       'Jet Airways ', 'Multiple Carriers ', 'Trujet'], dtype=object)

- 2.2 Date_of_Journey

In [111]:
pd.to_datetime(df['Date_of_Journey'],dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

- 2.3 Source

In [112]:
df.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

- 2.4 Destination

In [113]:
pd.to_datetime(df['Arrival_Time']).dt.time

0        01:10:00
1        13:15:00
2        04:25:00
3        23:30:00
4        21:35:00
           ...   
10678    22:25:00
10679    23:20:00
10680    11:20:00
10681    14:10:00
10682    19:15:00
Name: Arrival_Time, Length: 10683, dtype: object

- 2.5 Dep_Time

In [114]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


- 2.6 Arrival_Time

In [115]:
df.Arrival_Time.unique()

array(['01:10 22 Mar', '13:15', '04:25 10 Jun', ..., '06:50 10 Mar',
       '00:05 19 Mar', '21:20 13 Mar'], dtype=object)

In [116]:
(
    df
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    .str.split(" ",n=1)
    .str.get(0)
    .unique()
)

array(['01:10', '04:25', '10:25', '05:05', '09:20', '19:15', '12:35',
       '13:20', '08:15', '03:35', '19:00', '11:05', '10:55', '00:15',
       '02:45', '12:00', '01:30', '16:45', '09:25', '04:40', '11:25',
       '14:25', '14:35', '09:05', '01:35', '10:05', '11:10', '07:45',
       '00:55', '23:35', '00:40', '18:50', '07:55', '18:15', '13:15',
       '09:45', '06:50', '15:15', '07:40', '02:20', '18:30', '19:45',
       '19:50', '01:00', '00:05', '16:20', '00:30', '00:50', '13:45',
       '09:15', '07:10', '08:55', '04:45', '05:35', '09:00', '10:45',
       '22:10', '22:40', '05:25', '02:15', '20:20', '06:35', '01:20',
       '19:10', '10:50', '23:25', '12:15', '10:40', '23:15', '22:55',
       '09:10', '18:10', '00:25', '20:45', '21:05', '23:00', '00:45',
       '04:35', '18:00', '12:30', '02:10', '00:20', '23:20', '12:25',
       '22:35', '20:25', '00:35', '16:50', '09:30', '12:05', '10:35',
       '15:05', '18:40', '09:35', '08:50', '16:40', '19:40', '16:10',
       '21:20', '07:

- 2.7 Duration

In [117]:
df.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [118]:
a=(
    df
    .Duration
    .str.extract('(\d+)')
)
a

,0
0,2
1,7
2,19
3,5
4,4
...,...
10678,2
10679,2
10680,3
10681,2


In [119]:
a=df['Duration'].str.split(" ",expand=True)

hour1=a[0].str.extract('(\d+)')
hour2=a[1].str.extract('(\d+)')

In [120]:
a=df['Duration'].str.split(" ",expand=True)

hour1=a[0].str.extract('(\d+)')
hour2=a[1].str.extract('(\d+)')
hour1 = hour1.fillna(0).astype(int)
hour2 = hour2.fillna(0).astype(int)
time=hour1*60+hour2
time

,0
0,170
1,445
2,1140
3,325
4,285
...,...
10678,150
10679,155
10680,180
10681,160


In [121]:
df['Duration']

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [122]:
def convert_to_minutes(duration):
    return(
        duration
        .drop(index=6474)
        .str.split(" ",expand=True)
        .set_axis(['hours','minutes'],axis=1)
        .assign(
            hours=lambda df_:(
                df_
                .hours
                .str.replace('h'," ")
                .astype(int)
                .mul(60)),
            minutes=lambda df_:(
                df_
                .minutes
                .str.replace('m'," ")
                .fillna(0)
                .astype(int))
            
        )
        .sum(axis=1)
    )
    

- Total Stops

In [123]:
(
    df
    .Total_Stops
    .str.replace('non-stop', '0')
    .str.replace(' stops?',"",regex=True)
    .unique()
)

array(['0', '2', '1', '3', nan, '4'], dtype=object)

- Additional_Info

In [124]:
df.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

- Cleaning Operations

In [125]:
import pandas as pd

def clean(df):
    return (
        df
        .dropna()
        .drop_duplicates()
        .assign(**{
            col: df[col].str.strip()
            for col in df.select_dtypes(include='O').columns
        })
        .rename(columns=str.lower)
        .assign(
            airline=lambda df_: (
                df_
                .airline
                .str.replace("Premium economy", "")
                .str.replace("Business", "")
            ),
            date_of_journey=lambda df_: pd.to_datetime(df_.date_of_journey, dayfirst=True),
            source=lambda df_: df_.source.str.replace("Banglore", "Bengaluru"),
            destination=lambda df_: df_.destination.str.replace("Banglore", "Bengaluru"),
            dep_time=lambda df_: pd.to_datetime(df_.dep_time, dayfirst=True).dt.time,
            arrival_time=lambda df_: pd.to_datetime(df_.arrival_time, dayfirst=True).dt.time,
            duration=lambda df_: df_.duration.pipe(convert_to_minutes),
            total_stops=lambda df_: df_
                .total_stops
                .str.replace('non-stop', '0')
                .str.replace(' stops?',"",regex=True)
                .pipe(lambda ser: pd.to_numeric(ser)),
            additional_info=lambda df_:df.Additional_Info.str.replace("No info", "No Info")
            

            
        )
        .drop(columns='route')
    )


In [126]:
cleaned_data=clean(df)
cleaned_data

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,IndiGo,2019-03-24,Bengaluru,New Delhi,22:20:00,01:10:00,170.0,0,No Info,3897
1,Air India,2019-05-01,Kolkata,Bengaluru,05:50:00,13:15:00,445.0,2,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140.0,2,No Info,13882
3,IndiGo,2019-05-12,Kolkata,Bengaluru,18:05:00,23:30:00,325.0,1,No Info,6218
4,IndiGo,2019-03-01,Bengaluru,New Delhi,16:50:00,21:35:00,285.0,1,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Bengaluru,19:55:00,22:25:00,150.0,0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Bengaluru,20:45:00,23:20:00,155.0,0,No Info,4145
10680,Jet Airways,2019-04-27,Bengaluru,Delhi,08:20:00,11:20:00,180.0,0,No Info,7229
10681,Vistara,2019-03-01,Bengaluru,New Delhi,11:30:00,14:10:00,160.0,0,No Info,12648


In [127]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10462 entries, 0 to 10682
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   airline          10462 non-null  object        
 1   date_of_journey  10462 non-null  datetime64[ns]
 2   source           10462 non-null  object        
 3   destination      10462 non-null  object        
 4   dep_time         10462 non-null  object        
 5   arrival_time     10462 non-null  object        
 6   duration         10461 non-null  float64       
 7   total_stops      10462 non-null  int64         
 8   additional_info  10462 non-null  object        
 9   price            10462 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 1.1+ MB


In [128]:
cleaned_data.dtypes

airline                    object
date_of_journey    datetime64[ns]
source                     object
destination                object
dep_time                   object
arrival_time               object
duration                  float64
total_stops                 int64
additional_info            object
price                       int64
dtype: object

- Splitting Data

In [129]:
flights_cleaned_data=cleaned_data.sample(1000)

x=flights_cleaned_data.drop(columns='price')
y=flights_cleaned_data.price

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=42)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
print(x_val.shape,y_val.shape)

(640, 9) (640,)
(200, 9) (200,)
(160, 9) (160,)


- Exporting Subsets

In [94]:
def export(x,y,name):
    file_name= f"{name}.csv"
    path=os.path.join(DATA_DIR,file_name)

    x.join(y).to_csv(path,index=False)
    return pd.read_csv(path)

In [95]:
export(x_train,y_train,"train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,IndiGo,2019-06-12,Delhi,Cochin,10:35:00,01:30:00,895.0,1,No Info,5883
1,IndiGo,2019-06-18,Bengaluru,Delhi,10:10:00,13:00:00,170.0,0,No Info,3943
2,Jet Airways,2019-05-15,Kolkata,Bengaluru,09:35:00,10:55:00,1520.0,1,No Info,14151
3,Jet Airways,2019-06-03,Delhi,Cochin,19:45:00,19:00:00,1395.0,1,In-flight meal not included,10262
4,IndiGo,2019-05-18,Kolkata,Bengaluru,15:30:00,18:05:00,155.0,0,No Info,4804
...,...,...,...,...,...,...,...,...,...,...
635,Jet Airways,2019-06-27,Delhi,Cochin,20:55:00,12:35:00,940.0,1,In-flight meal not included,10262
636,Jet Airways,2019-06-03,Delhi,Cochin,17:30:00,19:00:00,1530.0,1,No Info,14714
637,Jet Airways,2019-03-21,Delhi,Cochin,17:30:00,18:50:00,1520.0,1,In-flight meal not included,5963
638,Multiple carriers,2019-05-09,Delhi,Cochin,09:00:00,21:00:00,720.0,1,No Info,13727


In [96]:
export(x_val,y_val,"val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Multiple carriers,2019-06-09,Delhi,Cochin,16:00:00,01:30:00,570.0,1,No Info,13587
1,Jet Airways,2019-06-12,Kolkata,Bengaluru,06:30:00,12:00:00,330.0,1,No Info,13626
2,Multiple carriers,2019-05-21,Delhi,Cochin,08:45:00,21:00:00,735.0,1,No Info,7485
3,Multiple carriers,2019-06-09,Delhi,Cochin,09:00:00,21:00:00,720.0,1,No Info,7408
4,Jet Airways,2019-05-09,Delhi,Cochin,05:30:00,12:35:00,425.0,2,In-flight meal not included,15544
...,...,...,...,...,...,...,...,...,...,...
155,SpiceJet,2019-05-12,Bengaluru,Delhi,09:30:00,12:20:00,170.0,0,No check-in baggage included,4319
156,SpiceJet,2019-03-27,Kolkata,Bengaluru,22:20:00,00:40:00,140.0,0,No Info,3847
157,Jet Airways,2019-03-27,Delhi,Cochin,19:10:00,19:45:00,1475.0,2,In-flight meal not included,6643
158,Jet Airways,2019-05-15,Kolkata,Bengaluru,06:30:00,16:20:00,590.0,1,In-flight meal not included,10844


In [97]:
export(x_test,y_test,"test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-12,Bengaluru,New Delhi,21:25:00,10:45:00,800.0,1,No Info,13817
1,Jet Airways,2019-03-18,Mumbai,Hyderabad,02:55:00,04:25:00,90.0,0,No Info,5678
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140.0,2,No Info,13882
3,Jet Airways,2019-03-21,Mumbai,Hyderabad,15:50:00,17:20:00,90.0,0,No Info,5678
4,Jet Airways,2019-03-27,Delhi,Cochin,07:05:00,04:25:00,1280.0,1,In-flight meal not included,6645
...,...,...,...,...,...,...,...,...,...,...
195,Air India,2019-05-09,Kolkata,Bengaluru,14:35:00,14:35:00,1440.0,1,No Info,6528
196,Multiple carriers,2019-06-24,Delhi,Cochin,06:00:00,21:00:00,900.0,1,No Info,7740
197,Jet Airways,2019-06-01,Delhi,Cochin,19:15:00,19:00:00,1425.0,1,In-flight meal not included,10262
198,Jet Airways,2019-06-27,Delhi,Cochin,21:25:00,12:35:00,910.0,2,In-flight meal not included,11507
